# Notebook 13: Non-Dimensional Scaling

When working with physical problems, the range of scales can cause numerical difficulties. For example, mantle convection involves:
- Velocities of ~10⁻⁹ m/s (tiny)
- Viscosities of ~10²¹ Pa·s (huge)
- Pressures of ~10⁹ Pa (large)

Non-dimensional (ND) scaling transforms the problem so all quantities are order-one, improving numerical conditioning and stability.

In this notebook you'll learn:
- Setting reference quantities for automatic scaling
- Solving Poisson and Stokes equations with ND scaling
- Understanding what happens under the hood
- Validating that ND and dimensional solutions match

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
os.environ["SYMPY_USE_CACHE"] = "no"

import underworld3 as uw
import numpy as np
import sympy

## Example 1: Heat Equation with ND Scaling

We'll solve a steady-state heat conduction problem with internal heat production (like radioactive heating) twice:
1. Without ND scaling (manual approach)
2. With ND scaling (automatic)

Both should give the same non-dimensional values in `.data`.

### Without ND Scaling (Manual approach)

First, we'll solve in non-dimensional form with no units (and no reference quantities set). The units are implicit and the user has to keep track of them correctly.

In [2]:
# Ensure clean slate - NO reference quantities, NO ND scaling
uw.reset_default_model()
uw.use_nondimensional_scaling(False)

# Create mesh (ND coordinates: 0 to 1)
mesh_manual = uw.meshing.StructuredQuadBox(elementRes=(8, 8))
x, y = mesh_manual.X

# Create temperature variable without units (will store ND values)
T_manual = uw.discretisation.MeshVariable("T_manual", mesh_manual, 1, degree=2, varsymbol=r"T_\textrm{man}")

# Set up Poisson solver for heat equation: ∇²T = Q
poisson_manual = uw.systems.Poisson(mesh_manual, u_Field=T_manual)
poisson_manual.constitutive_model = uw.constitutive_models.DiffusionModel
poisson_manual.constitutive_model.Parameters.diffusivity = 1.0  # ND thermal diffusivity
poisson_manual.f = 2.0  # ND heat source

# Boundary conditions (ND values - cold boundaries)
poisson_manual.add_dirichlet_bc(0.0, "Bottom")
poisson_manual.add_dirichlet_bc(0.0, "Top")

# Solve
poisson_manual.solve()

# Store ND solution
T_manual_data = np.copy(T_manual.data)

print(f"Manual ND: T_max = {T_manual.max()}")

Structured box element resolution 8 8
Manual ND: T_max = 0.2500029213877409


### With ND Scaling (Automatic based on reference scales)

Now solve the same problem with ND scaling. We'll use simple reference scales (L₀=1 m, V₀=1 m/s) so the numbers are the same as far as the numerical solvers are concerned, but underworld will track the units for you and will help you convert from one to another.

In [3]:
# Reset and set up ND scaling
uw.reset_default_model()
model = uw.get_default_model()

# Set reference quantities (triggers unit requirement everywhere)
model.set_reference_quantities(
    domain_depth=uw.quantity(1, "m"),  # L₀ = 1 m
    plate_velocity=uw.quantity(1, "m/s"),  # V₀ = 1 m/s (needed for system)
    mantle_viscosity=uw.quantity(1, "Pa*s"),  # η₀ = 1 Pa·s (needed for system)
    temperature_difference=uw.quantity(1, "K")  # T₀ = 1 K
)

# Create mesh
mesh_nd = uw.meshing.StructuredQuadBox(elementRes=(8, 8))
x, y = mesh_nd.X

# Create temperature variable WITH units
T_nd = uw.discretisation.MeshVariable("T_nd", mesh_nd, 1, degree=2, units="K", varsymbol=r"T_\textrm{ND}")

# Enable ND scaling
uw.use_nondimensional_scaling(True)

# Set up Poisson solver for heat equation
poisson_nd = uw.systems.Poisson(mesh_nd, u_Field=T_nd)
poisson_nd.constitutive_model = uw.constitutive_models.DiffusionModel
poisson_nd.constitutive_model.Parameters.diffusivity = 1.0
# Heat source WITH units: for ∇²T = Q, where T is in [K], Q must be in [K/m²]
# This represents internal heating (like radioactive decay)
# With L₀ = 1 m and T₀ = 1 K, dimensional Q = 2.0 K/m² gives ND value of 2.0
poisson_nd.f = uw.quantity(2.0, "K/m**2")

# Boundary conditions WITH units (cold boundaries at T = 0 K relative to reference)
poisson_nd.add_dirichlet_bc(uw.quantity(0.0, "K"), "Bottom")
poisson_nd.add_dirichlet_bc(uw.quantity(0.0, "K"), "Top")

# Solve
poisson_nd.solve()

# Store solution
T_nd_solution = np.copy(T_nd.data)

T_nd.min(), T_nd.max()

Structured box element resolution 8 8


(UWQuantity(0.0, 'kelvin'), UWQuantity(0.2500029213877409, 'kelvin'))

### Comparison

The solutions should match perfectly:

In [4]:
# Compute difference
difference = np.max(np.abs(T_manual_data - T_nd_solution))
relative_error = difference / np.max(np.abs(T_manual_data))

difference, relative_error

(0.0, 0.0)

The difference is at machine precision - as expected since we have only added unit-tracking to the problem, we haven't actually changed any values. You can see that the units are carried and the system will happily convert quantities for you. It is capable of more though.

Typically, we'd like to specify real-world values for all quantities. For a planetary modelling problem, lengths will be in thousands of km, viscosities will be in zetta Pascal seconds, and stresses will be in MPa or GPa. We can use that information to set reference quantities (e.g. reference viscosity is $10^{21}$ Pa.s) and specify all viscosities relative to that value. This avoids potential computational inaccuracy associated with having large forces applied to near-rigid materials and calculating tiny responses.


## Example 2: Stokes Flow with Realistic Scales

Now let's solve a lid-driven cavity flow problem with realistic mantle convection parameters. This demonstrates why ND scaling is important.

### Manual Non-Dimensionalization

First, solve with manual non-dimensionalisation (the traditional approach). We work directly with ND values and, when we have finished solving, we interpret the raw results by rescaling them to represent physical values. 

In [27]:
# Reset - NO ND scaling, manual non-dimensionalization
uw.reset_default_model()
uw.use_nondimensional_scaling(False)

# Create mesh (ND coordinates: 0 to 1)
resolution = 8
mesh_manual = uw.meshing.StructuredQuadBox(
    elementRes=(resolution, resolution),
    minCoords=(0.0, 0.0),
    maxCoords=(1.0, 1.0)
)

# Create variables without units (will store ND values)
v_manual = uw.discretisation.MeshVariable("v_manual", mesh_manual, 2, degree=2, varsymbol=r"v_\textrm{man}")
p_manual = uw.discretisation.MeshVariable("p_manual", mesh_manual, 1, degree=1, varsymbol=r"p_\textrm{man}")

# Set up Stokes solver with ND values
stokes_manual = uw.systems.Stokes(mesh_manual, velocityField=v_manual, pressureField=p_manual)
stokes_manual.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes_manual.constitutive_model.Parameters.viscosity = 1.0  # ND viscosity

# Lid-driven cavity: top moves with ND velocity = 1.0
stokes_manual.add_dirichlet_bc((1.0, 0.0), "Top")
stokes_manual.add_dirichlet_bc((0.0, 0.0), "Bottom")
stokes_manual.add_dirichlet_bc((sympy.oo, 0.0), "Left")  # v_x free, v_y = 0
stokes_manual.add_dirichlet_bc((sympy.oo, 0.0), "Right")  # v_x free, v_y = 0

Structured box element resolution 8 8


In [28]:
# Solve
stokes_manual.solve()

# Store ND solution
v_manual_data = np.copy(v_manual.data)
p_manual_data = np.copy(p_manual.data)

print(f"Manual ND: v_max = {v_manual.max()}, p_max = {p_manual.max()}")

Manual ND: v_max = (1.0, 3.53066915728927e-06), p_max = 2.4663425594768518e-05


In [29]:
uw.unwrap(stokes_manual.F1.sym, keep_constants=True)

Matrix([
[{ \uplambda \hspace{ 0.0029pt } }*({ \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 0,0}(N.x, N.y) + { \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 1,1}(N.x, N.y)) - { \hspace{ 0.0079pt } {p_\textrm{man}} }(N.x, N.y) + 2*{ \eta \hspace{ 0.0033pt } }*{ \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 0,0}(N.x, N.y),                                                                                                                                                      { \eta \hspace{ 0.0033pt } }*({ \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 0,1}(N.x, N.y) + { \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 1,0}(N.x, N.y))],
[                                                                                                                                                     { \eta \hspace{ 0.0033pt } }*({ \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 0,1}(N.x, N.y) + { \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 1,0}(N.x, N.y)), { \uplambda \hspace{ 0.0029pt } }*({ \hspace{ 0.0079pt } {v_\textrm{man}} }_{ 0,0}(N.x, N.y

In [30]:
uw.quantity(1e19, "Pa.s").to("GPa.s")

UWQuantity(10000000000.000002, 'gigapascal * second')

In [31]:
uw.non_dimensionalise(uw.quantity(1e19, "Pa.s"))

1e+19

### Automatic Non-Dimensionalization

Now solve the same problem using automatic ND scaling. We provide dimensional values, and the system converts to ND form automatically:

In [32]:
# Reset and set reference quantities
uw.reset_default_model()
model = uw.get_default_model()

# Define reference scales - same ND numbers as manual case
# We want V₀ = 1.0 (in ND form), so we just use reference=1.0 in compatible units
model.set_reference_quantities(
    domain_depth=uw.quantity(1000, "km"),            # Reference length = 1 m
    plate_velocity=uw.quantity(1, "cm/yr"),        # Reference velocity = 1 m/s
    mantle_viscosity=uw.quantity(1e19, "Pa*s")      # Reference viscosity = 1 Pa·s
)

# Create mesh (unit square like manual case, but now with units)
mesh_auto = uw.meshing.StructuredQuadBox(
    elementRes=(resolution, resolution),
    minCoords=(0,0),
    maxCoords=(uw.quantity(1000, "km"), uw.quantity(1000, "km"))
)

# Create variables WITH units
v_auto = uw.discretisation.MeshVariable("v_auto", mesh_auto, 2, degree=2, units="m/s", varsymbol=r"v_\textrm{auto}")
p_auto = uw.discretisation.MeshVariable("p_auto", mesh_auto, 1, degree=1, units="Pa", varsymbol=r"p_\textrm{auto}")

# Check what the scaling coefficients are
V0_val = v_auto.scaling_coefficient
P0_val = p_auto.scaling_coefficient

print(f"Scaling coefficients (with reference scales = 1.0):")
print(f"  V₀ = {V0_val:.6e} m/s")
print(f"  P₀ = {P0_val:.6e} Pa")

Structured box element resolution 8 8
Scaling coefficients (with reference scales = 1.0):
  V₀ = 3.168809e-10 m/s
  P₀ = 3.168809e+03 Pa


In [59]:
# Enable ND scaling
uw.use_nondimensional_scaling(True)

# Set up Stokes solver
stokes_auto = uw.systems.Stokes(mesh_auto, velocityField=v_auto, pressureField=p_auto)
stokes_auto.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes_auto.constitutive_model.Parameters.shear_viscosity_0 = uw.quantity(1e19, uw.units("Pa.s"))  # ND viscosity (reference = 1e19 Pa·s)

# Boundary conditions with reference units: To get ND value of 1.0,
# we provide dimensional value = 10.0 m/s (which equals 1.0 × V₀ when V₀ = 10 m/s)
stokes_auto.add_dirichlet_bc((uw.quantity(1.0, "cm/yr"), uw.quantity(0.0, "m/s")), "Top")
stokes_auto.add_dirichlet_bc((uw.quantity(0.0, "m/s"), uw.quantity(0.0, "m/s")), "Bottom")
stokes_auto.add_dirichlet_bc((sympy.oo, uw.quantity(0.0, "m/s")), "Left")
stokes_auto.add_dirichlet_bc((sympy.oo, uw.quantity(0.0, "m/s")), "Right")

# Solve
stokes_auto.petsc_options.setValue("ksp_monitor", None)
stokes_auto.petsc_options.setValue("snes_monitor", None)
stokes_auto.solve(verbose=False)

# Store solution (will be in ND form in .data)
v_auto_data = np.copy(v_auto.data)
p_auto_data = np.copy(p_auto.data)

print(f"Automatic scaling: v_max = {np.nanmax(np.abs(v_auto_data)):.6e}, p_max = {np.nanmax(np.abs(p_auto_data)):.6e}")

  0 SNES Function norm 5.630056026718e+00
    Residual norms for Solver_119_ solve.
    0 KSP Residual norm 9.085127539351e+00
    1 KSP Residual norm 4.402200861821e-01
  1 SNES Function norm 2.650874524246e-02
    Residual norms for Solver_119_ solve.
    0 KSP Residual norm 4.275011810652e-02
    1 KSP Residual norm 1.994137192605e-03
  2 SNES Function norm 1.116095545385e-04
Automatic scaling: v_max = 1.000000e+00, p_max = 1.727671e-01


In [63]:
v_auto.array[...].max()

UWQuantity(3.1688087814028947e-10, 'meter / second')

In [66]:
uw.function.evaluate(v_auto, v_auto.coords)[0:10]

array([[[ 1.96418936e+27,  2.53413638e+19]],

       [[ 5.89256806e+27,  7.79293825e+18]],

       [[ 9.82094670e+27, -1.35569890e+20]],

       [[ 1.37493250e+28, -1.83743347e+20]],

       [[ 1.76777030e+28, -1.21772533e+20]],

       [[ 2.16060807e+28, -1.15101784e+20]],

       [[ 2.55344586e+28, -7.18617339e+19]],

       [[ 2.94628370e+28, -4.65858301e+19]],

       [[ 1.96418937e+27, -2.72321858e+19]],

       [[ 5.89256817e+27, -5.54905502e+19]]])

### Validation: Same ND Values

Both approaches solve the same ND problem. Let's verify that `.data` contains identical ND values:

In [ ]:
# Compare ND values in .data arrays
v_diff = np.max(np.abs(v_manual_data - v_auto_data))
v_rel_error = v_diff / np.max(np.abs(v_manual_data))

p_diff = np.max(np.abs(p_manual_data - p_auto_data))
p_rel_error = p_diff / (np.max(np.abs(p_manual_data)) + 1e-15)

print(f"Velocity difference: {v_diff:.3e} (relative: {v_rel_error:.3e})")
print(f"Pressure difference: {p_diff:.3e} (relative: {p_rel_error:.3e})")

v_diff, v_rel_error, p_diff, p_rel_error

The values of all raw values in the uw data containers are identical whether we scale by hand or automatically, but the view we have of the data in the scaled case is more flexible - it can be converted to any units of the correct dimensions. The `array` property of the variable reflects units and scaling, whereas the `data` property is a view into the raw numbers that are used by PETSc. 

In [ ]:
v_manual.array[0:10].squeeze()

In [ ]:
v_auto.array[0:10].squeeze()

In [ ]:
v_manual.data[0:10]

In [ ]:
v_auto.data[0:10]

### Behind the scenes

What just happened:

**Manual ND (Traditional Approach):**
 - User manually non-dimensionalizes the problem
 - Sets up equations with ND values (viscosity=1.0, BC=1.0, etc.)
 - `.data` contains ND values directly and `.array` has the same values
 - User must track what the values mean physically

**Automatic ND (Underworld3 System):**

 - User provides reference quantities and units
 - System derives scaling coefficients (V₀, P₀, etc.)
 - User provides in physical units, the system automatically non-dimensionalises them when they are used
 - `.data` contains the **same ND values** as manual approach, but the `.array` is in convenient units
 - System tracks physical meaning (dimensionality of quantities) automatically

Both cases give PETSc identical ND matrices to solve. The difference is whether the user or the system manages the unit tracking and scaling.

This is why `.data` arrays are identical - they both contain the non-dimensional values that PETSc actually works with !


**Note:** It would be a fair criticism to say that this is not a particularly taxing test for the reason that this is a lid driven flow with velocity entirely controlled by the surface value. We will see more complicated examples in the next Notebook.

## Things to try 

Exercises to explore:

```python
# 1. Try different reference quantities
model.set_reference_quantities(
    domain_depth=uw.quantity(100, "km"),  # Lithosphere scale
    plate_velocity=uw.quantity(1, "cm/year"),
    mantle_viscosity=uw.quantity(1e23, "Pa*s")  # Lithosphere viscosity
)

# 2. Check scaling coefficients
v.scaling_coefficient  # What is V₀?
p.scaling_coefficient  # What is P₀?

# 3. Validate with different resolutions
# Do the solutions still match with elementRes=(16, 16)?
```